In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# /content/ASL

In [1]:
import numpy
x_train = numpy.load('/content/x_train.npy')
y_train = numpy.load('/content/y_train.npy')

In [1]:
!git clone https://github.com/ASLVA/ASL.git

Cloning into 'ASL'...
remote: Enumerating objects: 18426, done.
remote: Counting objects: 100% (3325/3325), done.
remote: Compressing objects: 100% (3319/3319), done.
remote: Total 18426 (delta 5), reused 3278 (delta 1), pack-reused 15101
Receiving objects: 100% (18426/18426), 195.20 MiB | 42.39 MiB/s, done.
Resolving deltas: 100% (682/682), done.


In [1]:
# load train images and preprocess them using inbuilt functions

import cv2
import numpy
import glob

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array

def fetch_image(file_path):
    path = glob.glob(file_path)
    imagenames_list = []
    image_labels = []
    for folder in path:
      for f in glob.glob(folder+"/*.*"):
        image_name = f.split('/')[6]
        image_labels.append(image_name)
        imagenames_list.append(f)

    print(len(imagenames_list))

    read_images = []     
    i = 0   
    for image in imagenames_list:
      i += 1
    
      read_images.append(load_img(image,color_mode='rgb', target_size=(224, 224)))
    return read_images, image_labels

In [2]:
# prepare x_train and y_train

import tensorflow as tf
def prepare_images(image_labels, read_images, test=None):
  image_data = []
  for i in range(len(read_images)):
    img_data = tf.keras.utils.img_to_array(read_images[i])
    # img_data = np.expand_dims(img_data, axis = 0)
    img_data = preprocess_input(img_data)

    image_data.append(img_data)
  x = numpy.asarray(image_data)

  print(x.shape)
  if test:
    y_train = []
    y_label = list(set(image_labels))
    for i in image_labels:
      y_train.append(y_label.index(f"{i}"))

    y = tf.keras.utils.to_categorical(y_train, num_classes=26) 
  else:
    y_train = []
    for i in range(len(image_labels)):
      y = ord(image_labels[i]) - 65
      y_train.append(y)

    y = tf.keras.utils.to_categorical(y_train, num_classes=26)

  return x, y

In [3]:
read_train_img, train_img_labels = fetch_image("/content/ASL/data/train/*")
x_train, y_train = prepare_images(train_img_labels, read_train_img)


2600
(2600, 224, 224, 3)


In [5]:
read_test_img, test_img_labels = fetch_image("/content/ASL/data/test_sample/test1/*")
x_test, y_test = prepare_images(test_img_labels, read_test_img, True)

260
(260, 224, 224, 3)


In [8]:
numpy.save('x_train', x_train)
numpy.save('y_train', y_train)

In [9]:
numpy.save('x_test', x_test)
numpy.save('y_test', y_test)

In [6]:
# split train dataset into train dataset and validation dataset

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

In [7]:
def fit(x,y,batch,epoch,x1,y1,model):
  model.fit(x, y, batch_size=batch, epochs=epoch, validation_data=(x1,y1))
  return model

In [8]:
# extract weights from all the layers in first model of the cascade

def get_weights(model):
  weights = []
  for layer in model.layers:
      wts = layer.get_weights()
      weights.append(wts)
  return weights

In [9]:
# Accuracy of model 1 in cascade 1, initialized with imagenet weights, trained on ASL dataset
import numpy as np

import warnings
warnings.filterwarnings('ignore')
# Accuracy
from sklearn import metrics
def eval_metrics(x,y,model):
  y_predicted = model.predict(x)
  metric = metrics.classification_report(np.argmax(y, axis=1), np.argmax(y_predicted, axis=1))
  return metric

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
# y_pred1 = eval_metrics(x_test,y_test,cascade_model_1a_gen)
# print(y_pred1)

In [ ]:
def get_max_prob(y_pred):
  y_pred_list = []
  for i in y_pred:
    max_value = max(i)
    max_index = numpy.where(i == max_value)
    y_pred_list.append((max_value, max_index))

  return y_pred_list


First Cascade VGG16-VGG16

In [10]:
# add additional layers to the base model

CLASSES = 26
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Flatten
from keras import layers, models
from keras.callbacks import ModelCheckpoint
from keras.models import  load_model

def get_model(base_model):
  
  model = models.Sequential()
  model.add(base_model)
  model.add(layers.Flatten())
  model.add(layers.Dense(4096, activation='relu'))  
  model.add(layers.Dense(CLASSES, activation='softmax'))

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])
  
  return model

In [ ]:
# base model VGG1, Trained on ASL, Initialized with ImageNet

IMG_SIZE = (224,224)

def get_vgg():

  base_model_vgg = VGG16(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_vgg.layers:
    layer.trainable = True

  return base_model_vgg

In [ ]:
base_model_vgg = get_vgg()
cascade_vgg_1 = get_model(base_model_vgg)

In [ ]:
# # generalize this model to test on digits dataset, by training only the last layer freezing all other layers

# def get_gen_model(model):
  
#   model.pop()
#   model.add(layers.Dense(10, activation='softmax'))

#   model.compile(
#     loss='categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy'])
  
#   return model

In [ ]:
# cascade_model_1a_gen = get_gen_model(cascade_model_1a)

# for layer in cascade_model_1a_gen.layers:
#   if layer.output_shape == (None,10):
#     layer.trainable = True
#   else:
#     layer.trainable = False

# for layer in cascade_model_1a_gen.layers:
#   print(layer.name)
#   print(layer.trainable)

In [ ]:
cascade_vgg_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 26)                106522    
                                                                 
Total params: 117,585,754
Trainable params: 117,585,754
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cascade_vgg_1 = fit(x_train, y_train, 50, 10, x_valid,y_valid,cascade_vgg_1)

Epoch 1/10
4/4 [==============================] - 23s 4s/step - loss: 9.8620 - accuracy: 0.0385 - val_loss: 118.2850 - val_accuracy: 0.0385
Epoch 2/10
4/4 [==============================] - 3s 715ms/step - loss: 96.2342 - accuracy: 0.0330 - val_loss: 3.5240 - val_accuracy: 0.0513
Epoch 3/10
4/4 [==============================] - 3s 711ms/step - loss: 11.4036 - accuracy: 0.0330 - val_loss: 3.2768 - val_accuracy: 0.0385
Epoch 4/10
4/4 [==============================] - 3s 708ms/step - loss: 7.6945 - accuracy: 0.0275 - val_loss: 3.2701 - val_accuracy: 0.0385
Epoch 5/10
4/4 [==============================] - 3s 701ms/step - loss: 3.2567 - accuracy: 0.0385 - val_loss: 3.2727 - val_accuracy: 0.0256
Epoch 6/10
4/4 [==============================] - 3s 703ms/step - loss: 3.2564 - accuracy: 0.0440 - val_loss: 3.2752 - val_accuracy: 0.0256
Epoch 7/10
4/4 [==============================] - 3s 708ms/step - loss: 5.0453 - accuracy: 0.0330 - val_loss: 4.3571 - val_accuracy: 0.0128
Epoch 8/10
4/4 [==

In [ ]:
y_pred_vgg1 = cascade_vgg_1.predict(x_test)

In [ ]:
y_pred_vgg1 = get_max_prob(y_pred_vgg1)

In [ ]:
eval_metrics(x_test, y_test, cascade_vgg_1)

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        10\n           1       0.00      0.00      0.00        10\n           2       0.00      0.00      0.00        10\n           3       0.04      1.00      0.07        10\n           4       0.00      0.00      0.00        10\n           5       0.00      0.00      0.00        10\n           6       0.00      0.00      0.00        10\n           7       0.00      0.00      0.00        10\n           8       0.00      0.00      0.00        10\n           9       0.00      0.00      0.00        10\n          10       0.00      0.00      0.00        10\n          11       0.00      0.00      0.00        10\n          12       0.00      0.00      0.00        10\n          13       0.00      0.00      0.00        10\n          14       0.00      0.00      0.00        10\n          15       0.00      0.00      0.00        10\n          16       0.00      0.00      0.00        10\n       

In [ ]:
wts_cascade_vgg = get_weights(cascade_vgg_1)

In [ ]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_vgg = get_vgg()

## iterate through base model layer names:

layer_count = 0
for layer in base_model_vgg.layers:
  if "pool" not in layer.name and "input" not in layer.name:
    weights = wts_cascade_vgg[0].pop(0)
    bias = wts_cascade_vgg[0].pop(0)

    # set weights and biases for base model 1
    base_model_vgg.layers[layer_count].set_weights([weights,bias])
  layer_count += 1

cascade_vgg_2 = get_model(base_model_vgg)

In [ ]:
cascade_vgg_2 = fit(x_train, y_train, 50, 10, x_valid,y_valid,cascade_vgg_2)

Epoch 1/10
4/4 [==============================] - 4s 790ms/step - loss: 5.2380 - accuracy: 0.0165 - val_loss: 3.2627 - val_accuracy: 0.0256
Epoch 2/10
4/4 [==============================] - 3s 713ms/step - loss: 3.2457 - accuracy: 0.0220 - val_loss: 3.2503 - val_accuracy: 0.0128
Epoch 3/10
4/4 [==============================] - 3s 722ms/step - loss: 3.1415 - accuracy: 0.0714 - val_loss: 3.4482 - val_accuracy: 0.0128
Epoch 4/10
4/4 [==============================] - 3s 723ms/step - loss: 3.0924 - accuracy: 0.0714 - val_loss: 3.2490 - val_accuracy: 0.0513
Epoch 5/10
4/4 [==============================] - 3s 801ms/step - loss: 2.9649 - accuracy: 0.1703 - val_loss: 4.2254 - val_accuracy: 0.0256
Epoch 6/10
4/4 [==============================] - 3s 735ms/step - loss: 3.0624 - accuracy: 0.1374 - val_loss: 3.1977 - val_accuracy: 0.0641
Epoch 7/10
4/4 [==============================] - 3s 728ms/step - loss: 2.9968 - accuracy: 0.1264 - val_loss: 3.5401 - val_accuracy: 0.0897
Epoch 8/10
4/4 [====

In [ ]:
import warnings
warnings.filterwarnings('ignore')
y_pred2 = eval_metrics(x_test,y_test,cascade_vgg_2)
print(y_pred2)

4/4 [==============================] - 1s 157ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        10
           8       0.10      1.00      0.19        10
           9       0.00      0.00      0.00        10
          19       0.00      0.00      0.00         0
          25       0.00      0.00      0.00         0

    accuracy                           0.10       100
   macro avg       0.01      0.08      0.02       100
weighted avg       0.01      0.10      0.02       100



In [ ]:
# cascade_model_1b.layers[0].get_weights()

In [ ]:
y_pred_vgg2 = cascade_vgg_2.predict(x_test)

In [ ]:
numpy.save('y_pred_vgg2 ', y_pred_vgg2 )

In [ ]:
# #[[[vgg_values]],[flatten],[Dense],[softmax]]
# len(weights_list[0])  #----> len(VGG)
# len(weights_list[0][0]) # First layer of VGG shape --> (3,3,3,64)  --- > channels R, G, B
# #3 
# len(weights_list[0][0][0]) # First layer of VGG shape --> (3,3,3,64)  --- > channel R, last index if it changes to 1 gives G, if its 2 it gives B

In [ ]:
##### R values #####
# []
# 1st sublist[R] - 1st sublist = first row first column of all 64 filters
# 1st sublist[R] - 2nd sublist = first row, second column of all 64 filters
# 1st sublist[R]  - 3rd sublist = first row ,third column of all 64 filters

In [ ]:
# []
# 2st sublist[R] - 1st sublist = second row first column of all 64 filters
# 2st sublist[R] - 2nd sublist = second row, second column of all 64 filters
# 2st sublist[R]  - 3rd sublist = second row ,third column of all 64 filters

In [ ]:
# []
# 3st sublist[R] - 1st sublist = third row first column of all 64 filters
# 3st sublist[R] - 2nd sublist = third row, second column of all 64 filters
# 3st sublist[R]  - 3rd sublist = third row ,third column of all 64 filters

In [ ]:
# First row of the R channels filters 
# weights_list[0][0][0][0]

In [ ]:
# first sublist is R, second sublist is G and third sublist is B, 
# each sublist within shows 1st row, 2nd row and 3rd row

# weights_1 = weights_list[0][0]
# weights_1

In [ ]:
## model [0][0] - 1st layer weights      ## base_model_1 [0][0] - inputs
## model [0][1] - 1st layer bias         ## base_model_1 [0][1] - 1st layer weights + bias,

## model [0][2] - 2nd layer weights      ## base_model_1 [0][2] - 2nd layer weights + bias, 2,3
## model [0][3] - 2nd layer bias         
                                         ## base_model_1 [0][3] - maxpool

## model [0][4] - 3rd layer weights      ## base_model_1 [0][4] - 3rd layer weights + bias, 4,5
## model [0][5] - 3rd layer bias         

## model [0][6] - 4th layer weights      ## base_model_1 [0][4] - 4th layer weights + bias, 6,7
## model [0][7] - 4th layer bias 

## model [0][8] - 4th layer weights      ## base_model_1 [0][5] - 4th layer weights + bias, 8,9
## model [0][9] - 4th layer bias 

In [ ]:
## base_model [0][0] - 1st layer weights      ## base_model_1 [0][0] - inputs
## base_model [0][1] - 1st layer bias         ## base_model_1 [0][1] - 1st layer weights + bias, 0,1

## base_model [0][2] - 2nd layer weights      ## base_model_1 [0][2] - 2nd layer weights + bias, 2,3
## base_model [0][3] - 2nd layer bias         

## base_model [0][4] - 3rd layer weights      ## base_model_1 [0][3] - 3rd layer weights + bias, 4,5
## base_model [0][5] - 3rd layer bias         

## base_model [0][6] - 4th layer weights      ## base_model_1 [0][4] - 4th layer weights + bias, 6,7
## base_model [0][7] - 4th layer bias 

## base_model [0][8] - 4th layer weights      ## base_model_1 [0][5] - 4th layer weights + bias, 8,9
## base_model [0][9] - 4th layer bias 

In [ ]:
## <<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
# test_model = VGG16(
#     weights='imagenet', 
#     include_top=False, 
#     input_shape=(224,224,3)
# )

In [ ]:
# import numpy as np
# import random
# test_weights = np.random.rand(3,3,3,64)
# test_bias = np.random.rand(64)

In [ ]:
# test_weights

In [ ]:
# test_weights[:,:,:,0]

In [ ]:
# layer = test_model.layers[1]
# t_weights, t_bias = layer.get_weights()

In [ ]:
# test_model.layers[1].set_weights([test_weights,test_bias])

In [ ]:
# layer = test_model.layers[1]
# t1_weights, t1_bias = layer.get_weights()

In [ ]:
# t1_weights[:,:,:,0]

In [11]:
def get_gen_model(model):

  model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  
  for layer in model.layers:
    if layer.output_shape == (None,26):
      layer.trainable = True
    else:
      layer.trainable = False
  
  return model

In [12]:
# split the test dataset into sub-sets

from sklearn.model_selection import train_test_split

x_test, x_eval, y_test, y_eval = train_test_split(x_test, y_test, test_size=0.2, random_state=42)

Second Cascade - MobileNet-MobileNet

In [13]:
from keras.applications.mobilenet import MobileNet

def get_mnet():
  base_model_mnet = MobileNet(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_mnet.layers:
    layer.trainable = True

  return base_model_mnet


In [14]:
## build second base model MobileNet

base_model_mnet = get_mnet()
cascade_mnet_1 = get_model(base_model_mnet)

In [ ]:
# cascade_model_2a.summary()

In [16]:
cascade_mnet_1 = fit(x_train, y_train, 50, 10, x_valid, y_valid,cascade_mnet_1)

Epoch 1/10
37/37 [==============================] - 16s 399ms/step - loss: 3.1267 - accuracy: 0.7143 - val_loss: 1.2653 - val_accuracy: 0.6859
Epoch 2/10
37/37 [==============================] - 13s 350ms/step - loss: 0.0540 - accuracy: 0.9852 - val_loss: 0.5345 - val_accuracy: 0.8513
Epoch 3/10
37/37 [==============================] - 13s 353ms/step - loss: 0.0047 - accuracy: 0.9989 - val_loss: 0.4734 - val_accuracy: 0.8808
Epoch 4/10
37/37 [==============================] - 13s 362ms/step - loss: 8.1911e-04 - accuracy: 1.0000 - val_loss: 0.4392 - val_accuracy: 0.8846
Epoch 5/10
37/37 [==============================] - 13s 361ms/step - loss: 3.3726e-04 - accuracy: 1.0000 - val_loss: 0.4204 - val_accuracy: 0.8897
Epoch 6/10
37/37 [==============================] - 13s 351ms/step - loss: 2.8128e-04 - accuracy: 1.0000 - val_loss: 0.4051 - val_accuracy: 0.9038
Epoch 7/10
37/37 [==============================] - 13s 352ms/step - loss: 2.2495e-04 - accuracy: 1.0000 - val_loss: 0.3930 - val_

In [17]:
generalized_mnet1 = get_gen_model(cascade_mnet_1)

In [18]:
generalized_mnet1.fit(x_test,y_test, batch_size=50, epochs = 10)

Epoch 1/10
5/5 [==============================] - 4s 405ms/step - loss: 24.2244 - accuracy: 0.1442
Epoch 2/10
5/5 [==============================] - 1s 271ms/step - loss: 17.4578 - accuracy: 0.4663
Epoch 3/10
5/5 [==============================] - 1s 267ms/step - loss: 3.5511 - accuracy: 0.6394
Epoch 4/10
5/5 [==============================] - 1s 272ms/step - loss: 1.4314 - accuracy: 0.8125
Epoch 5/10
5/5 [==============================] - 1s 270ms/step - loss: 0.8624 - accuracy: 0.8558
Epoch 6/10
5/5 [==============================] - 1s 273ms/step - loss: 0.4165 - accuracy: 0.9327
Epoch 7/10
5/5 [==============================] - 1s 277ms/step - loss: 0.6595 - accuracy: 0.9519
Epoch 8/10
5/5 [==============================] - 1s 278ms/step - loss: 0.1671 - accuracy: 0.9663
Epoch 9/10
5/5 [==============================] - 1s 284ms/step - loss: 0.2607 - accuracy: 0.9760
Epoch 10/10
5/5 [==============================] - 1s 275ms/step - loss: 0.2478 - accuracy: 0.9712


In [21]:
eval_mnet1 = eval_metrics(x_eval,y_eval,generalized_mnet1)

In [22]:
eval_mnet1

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         3\n           1       0.00      0.00      0.00         4\n           2       0.00      0.00      0.00         1\n           3       0.00      0.00      0.00         2\n           4       0.00      0.00      0.00         2\n           5       0.00      0.00      0.00         1\n           6       0.00      0.00      0.00         1\n           8       0.00      0.00      0.00         3\n           9       1.00      0.50      0.67         2\n          10       0.00      0.00      0.00         3\n          11       0.00      0.00      0.00         4\n          12       0.00      0.00      0.00         1\n          13       0.00      0.00      0.00         2\n          14       0.00      0.00      0.00         2\n          15       0.00      0.00      0.00         2\n          16       0.00      0.00      0.00         3\n          17       0.00      0.00      0.00         2\n       

In [23]:
wts_cascade_mnet = get_weights(cascade_mnet_1)

In [24]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_mnet = get_mnet()

layer_count = 0
for layer in base_model_mnet.layers:
  if "pool" not in layer.name and "input" not in layer.name and "pad" not in layer.name and "relu" not in layer.name:
    if "bn" in layer.name:
      bn_weights = []
      for i in range(0,4):
        bn_weights.append(wts_cascade_mnet[0].pop(0))
      base_model_mnet.layers[layer_count].set_weights(bn_weights)
    else:
      weights = wts_cascade_mnet[0].pop(0)
      base_model_mnet.layers[layer_count].set_weights([weights])
  layer_count += 1

cascade_mnet_2 = get_model(base_model_mnet)

In [25]:
cascade_mnet_2= fit(x_train, y_train, 50, 10, x_valid, y_valid,cascade_mnet_2)

Epoch 1/10
37/37 [==============================] - 16s 376ms/step - loss: 1.9551 - accuracy: 0.5780 - val_loss: 15.5841 - val_accuracy: 0.2397
Epoch 2/10
37/37 [==============================] - 14s 376ms/step - loss: 0.0557 - accuracy: 0.9879 - val_loss: 4.2617 - val_accuracy: 0.5397
Epoch 3/10
37/37 [==============================] - 13s 360ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 1.1378 - val_accuracy: 0.8064
Epoch 4/10
37/37 [==============================] - 13s 348ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6611 - val_accuracy: 0.8667
Epoch 5/10
37/37 [==============================] - 13s 355ms/step - loss: 9.5955e-04 - accuracy: 1.0000 - val_loss: 0.5121 - val_accuracy: 0.8833
Epoch 6/10
37/37 [==============================] - 13s 348ms/step - loss: 7.4715e-04 - accuracy: 1.0000 - val_loss: 0.4511 - val_accuracy: 0.8910
Epoch 7/10
37/37 [==============================] - 13s 351ms/step - loss: 6.4819e-04 - accuracy: 1.0000 - val_loss: 0.4186 - val_acc

In [26]:
generalized_mnet2 = get_gen_model(cascade_mnet_2)

In [28]:
generalized_mnet2.fit(x_test,y_test, batch_size=10, epochs = 7)

Epoch 1/7
21/21 [==============================] - 3s 111ms/step - loss: 3.9569 - accuracy: 0.3558
Epoch 2/7
21/21 [==============================] - 2s 111ms/step - loss: 1.3304 - accuracy: 0.7115
Epoch 3/7
21/21 [==============================] - 2s 112ms/step - loss: 1.5340 - accuracy: 0.6827
Epoch 4/7
21/21 [==============================] - 2s 114ms/step - loss: 1.7957 - accuracy: 0.7788
Epoch 5/7
21/21 [==============================] - 2s 114ms/step - loss: 3.3346 - accuracy: 0.6779
Epoch 6/7
21/21 [==============================] - 2s 113ms/step - loss: 1.1380 - accuracy: 0.8317
Epoch 7/7
21/21 [==============================] - 2s 114ms/step - loss: 0.5526 - accuracy: 0.8798


In [29]:
eval_mnet2 = eval_metrics(x_eval,y_eval,generalized_mnet2)

In [30]:
eval_mnet2

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         3\n           1       0.00      0.00      0.00         4\n           2       0.00      0.00      0.00         1\n           3       0.00      0.00      0.00         2\n           4       0.00      0.00      0.00         2\n           5       0.00      0.00      0.00         1\n           6       0.00      0.00      0.00         1\n           8       0.00      0.00      0.00         3\n           9       0.00      0.00      0.00         2\n          10       0.00      0.00      0.00         3\n          11       0.00      0.00      0.00         4\n          12       0.00      0.00      0.00         1\n          13       0.00      0.00      0.00         2\n          14       0.00      0.00      0.00         2\n          15       0.00      0.00      0.00         2\n          16       0.60      1.00      0.75         3\n          17       0.00      0.00      0.00         2\n       

In [32]:
y_pred_mnet2 = generalized_mnet2.predict(x_eval)
y_pred_mnet2.shape

(52, 26)

In [33]:
numpy.save('y_pred_mnet2 ', y_pred_mnet2 )

Preprocess Test Dataset

In [ ]:
## preprocess test data

import cv2
import numpy
import glob

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array

path = glob.glob("/content/drive/MyDrive/NN DL Project/dataset/test1/*")

test_list = []
test_labels = []
for folder in path:
  for f in glob.glob(folder+"/*.JPG"):
    test_labels.append(folder[-1])
    test_list.append(f)

print(len(test_list))

test_images = []     
i = 0   
for image in test_list:
  i += 1
  test_images.append(load_img(image,color_mode='rgb', target_size=(224, 224)))

100


In [ ]:
# prepare x_test and y_test

import tensorflow as tf
test_data = []
for i in range(len(test_images)):
  img_data = tf.keras.utils.img_to_array(read_images[i])
  # img_data = np.expand_dims(img_data, axis = 0)
  img_data = preprocess_input(img_data)

  test_data.append(img_data)

x_test = numpy.asarray(test_data)

y_test = test_labels
# for i in range(len(test_labels)):
#   y = ord(test_labels[i]) - 65
#   y_test.append(y)

y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

Third Cascade - ResNet50-ResNet50

In [ ]:
# train third pre-trained model Resnet
from keras.applications.resnet import ResNet50

def get_resNet():
  base_model_res = ResNet50(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_res.layers:
    layer.trainable = True
  
  return base_model_res

In [ ]:
base_model_resnet = get_resNet()
cascade_resnet_1 = get_model(base_model_resnet)

In [ ]:
cascade_resnet_1 = fit(x_train, y_train, 20, 1, x_valid, y_valid, cascade_resnet_1)

10/10 [==============================] - 14s 552ms/step - loss: 218.9130 - accuracy: 0.0714 - val_loss: 3578847.2500 - val_accuracy: 0.0641


In [ ]:
wts_cascade_resnet = get_weights(cascade_resnet_1)

In [ ]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_resnet = get_resNet()



layer_count = 0
for layer in base_model_resnet.layers:
  if "pool" not in layer.name and "input" not in layer.name and "pad" not in layer.name and "relu" not in layer.name and "add" not in layer.name and "out" not in layer.name:
    if "bn" in layer.name:
      bn_weights = []
      for i in range(0,4):
        bn_weights.append(wts_cascade_resnet[0].pop(0))
      base_model_resnet.layers[layer_count].set_weights(bn_weights)
    else:
      weights = wts_cascade_resnet[0].pop(0)
      bias = wts_cascade_resnet[0].pop(0)
      base_model_resnet.layers[layer_count].set_weights([weights,bias])
  layer_count += 1

cascade_resnet_2 = get_model(base_model_resnet)

In [ ]:
cascade_resnet_2 = fit(x_train, y_train, 20, 1, x_valid, y_valid, cascade_resnet_2)

10/10 [==============================] - 9s 412ms/step - loss: 43.1737 - accuracy: 0.0330 - val_loss: 3457251.2500 - val_accuracy: 0.0128


In [ ]:
y_pred_resnet2 = cascade_resnet_2.predict(x_test)
y_pred_resnet2.shape

(260, 26)

In [ ]:
numpy.save('y_pred_resnet2', y_pred_resnet2)

In [ ]:
import numpy
y_pred_vgg2 = numpy.load('/content/y_pred_vgg2 .npy')
y_pred_mnet2 = numpy.load('/content/y_pred_mnet2 .npy')
y_pred_resnet2 = numpy.load('/content/y_pred_resnet2.npy')

In [ ]:
len(y_pred_mnet2[0])

26

In [ ]:
from numpy.lib.function_base import average
softmax = []
for vgg, mnet, resnet in zip(y_pred_vgg2, y_pred_mnet2, y_pred_resnet2):
  average_prop = []
  for i in range(len(vgg)):
    avg = (vgg[i] + mnet[i] + resnet[i]) / 3
    average_prop.append(avg)
  max_value = max(average_prop)
  softmax.append((max_value,average_prop.index(max_value)))

In [ ]:
softmax

In [ ]:
list(set(test_img_labels))[4]

'More'